In [1]:
import json
from pprint import pprint
import glob, os
import re

data_path = "C:\\Users\\Yurtsever\\baby-activity-recog-new\\all_jsons"
root_path = "C:\\Users\\Yurtsever\\baby-activity-recog-new\\combined_jsons_19"

activity_list =["arching_back", "head_banging", "kicking_legs", "rubbing_eye", "stretching", "sucking_fingers"]

In [ ]:
# This notebook combines JSON outputs of OpenPose HPE.
# JSON outputs have excess information for our application.
# We need only detected keypoint coordinates - which are x and y values on the video frames.
# This notebook does exactly this - extracting only x and y values and writing it to another file.
# Later, these seperate txt outputs will be merged for train_test_split to train the model.

In [2]:
activity_nums = 6

all_jsons = len(os.listdir(data_path))

print ("All JSON Files: " + str(all_jsons) )

All JSON Files: 13795


In [3]:
def read_json_file(vid_file_name):
    os.chdir(data_path)
    
    print ("Reading file: " + str(vid_file_name) )
    with open(file) as data_file: 
        data = json.load(data_file)
        
        if len(data["people"]) > 1:
            pprint("More than one detection in file, check the noise:")
            # openpose_2person_count += 1
            print (file)
            
        # holds x and y values of the current frame - later will be appended to kps.
        frame_kps = []     
        try:
            pose_keypoints = data['people'][0]['pose_keypoints_2d']
            # print(data['people'][0]['pose_keypoints_2d'])
        except:
            print("JSON file cannot read: " + file)
            
        #print(data['people'][0]['pose_keypoints_2d'])
        #loop through 25 pose keypoints (total = 75, 25x3 (x, y and accuracy))
        # we will loop through for 19 keypoints - ignoring foots.
        # total = 19*2 = 57 
        j = 0
        for i in range(38): # 50
            frame_kps.append(pose_keypoints[j])
            j += 1
            if ((j+1) % 3 == 0):
                j += 1
        #kps.append(frame_kps)
        return frame_kps

In [4]:
#Now we have kps, a list of lists, that includes the x and y positions of all 18 keypoints, for all frames in the frameset
# So a list of length frameset.length, with each element being a 36 element long list.
#Next, we simply loop through kps, writing the contents to a text file, where each sub list is a new line.
#At this point, there is no overlap, and datasets are all of varying length

def write_frames_txt(vid_file_name):
    os.chdir(root_path)
    
    # create filename for combined frames
    frame_idx = vid_file_name.index(")") + 1
    output_file = str (vid_file_name[:frame_idx] + ".txt")
    #output_file_name = str(output_file) + str (".txt")
    
    with open(output_file, "w") as text_file:
        for i in range(len(kps)):
            for j in range(38): #50
                text_file.write('{}'.format(kps[i][j]))
                if j < 37: #49
                    text_file.write(',')
            text_file.write('\n')

In [5]:
os.chdir(data_path)

# arching_back (2).mp4_1_keypoints.json
# first video idx starts with 2
video_number = 2

last_frame_name = ""

# kps is a list of pose keypoints in each frame, 
# where kps[0] is the x position of kp0, kps[1] is the y position of kp0 etc
kps = []

for file in sorted(glob.glob("*.json"), key = os.path.getmtime):
    vid_file_name = file   
    video_iterator = re.search("^.*?\([^\d]*(\d+)[^\d]*\).*$", vid_file_name)
    
    # arching_back (2) --> holds number inside parentheses
    video_iter_number = int(video_iterator.group(1)) 
    
    # ayni videodaki farkli frame jsonlarini birlestiriyoruz.
    # while video_number == video_iter_number:
    if (video_number == video_iter_number):
        last_frame_name = vid_file_name
        current_frame_json = read_json_file(vid_file_name)
        
        # append frame to frame list of current video.
        kps.append(current_frame_json)
    
    # yeni videoya geciyor. frame bilgilerini yazdirip kps'yi temizle.
    else:
        write_frames_txt(last_frame_name)
        kps.clear()
        
        #else'e dusen ilk frame bilgisini kaybetmemek icin manuel cagirilir.
        current_frame_json = read_json_file(vid_file_name)       
        kps.append(current_frame_json)
        
        video_number = video_iter_number
    
# son video frameleri else'e duser. manuel olarak cagiriyoruz.
write_frames_txt(last_frame_name)

Reading file: arching_back (2).mp4_1_keypoints.json
Reading file: arching_back (2).mp4_2_keypoints.json
Reading file: arching_back (2).mp4_3_keypoints.json
Reading file: arching_back (2).mp4_4_keypoints.json
Reading file: arching_back (2).mp4_5_keypoints.json
Reading file: arching_back (2).mp4_6_keypoints.json
Reading file: arching_back (2).mp4_7_keypoints.json
'More than one detection in file, check the noise:'
arching_back (2).mp4_7_keypoints.json
Reading file: arching_back (2).mp4_8_keypoints.json
Reading file: arching_back (2).mp4_9_keypoints.json
Reading file: arching_back (2).mp4_10_keypoints.json
Reading file: arching_back (2).mp4_11_keypoints.json
Reading file: arching_back (2).mp4_12_keypoints.json
Reading file: arching_back (2).mp4_13_keypoints.json
Reading file: arching_back (2).mp4_14_keypoints.json
Reading file: arching_back (2).mp4_15_keypoints.json
Reading file: arching_back (2).mp4_16_keypoints.json
Reading file: arching_back (2).mp4_17_keypoints.json
Reading file: arch

Reading file: arching_back (6).mp4_91_keypoints.json
'More than one detection in file, check the noise:'
arching_back (6).mp4_91_keypoints.json
Reading file: arching_back (6).mp4_92_keypoints.json
'More than one detection in file, check the noise:'
arching_back (6).mp4_92_keypoints.json
Reading file: arching_back (6).mp4_93_keypoints.json
Reading file: arching_back (6).mp4_94_keypoints.json
Reading file: arching_back (6).mp4_95_keypoints.json
Reading file: arching_back (6).mp4_96_keypoints.json
Reading file: arching_back (6).mp4_97_keypoints.json
Reading file: arching_back (6).mp4_98_keypoints.json
Reading file: arching_back (6).mp4_99_keypoints.json
Reading file: arching_back (6).mp4_100_keypoints.json
Reading file: arching_back (6).mp4_101_keypoints.json
Reading file: arching_back (6).mp4_102_keypoints.json
Reading file: arching_back (6).mp4_103_keypoints.json
'More than one detection in file, check the noise:'
arching_back (6).mp4_103_keypoints.json
Reading file: arching_back (6).mp

Reading file: arching_back (14).mp4_3_keypoints.json
Reading file: arching_back (14).mp4_4_keypoints.json
Reading file: arching_back (14).mp4_5_keypoints.json
Reading file: arching_back (14).mp4_6_keypoints.json
Reading file: arching_back (14).mp4_7_keypoints.json
Reading file: arching_back (14).mp4_8_keypoints.json
Reading file: arching_back (14).mp4_9_keypoints.json
Reading file: arching_back (14).mp4_10_keypoints.json
Reading file: arching_back (14).mp4_11_keypoints.json
Reading file: arching_back (14).mp4_12_keypoints.json
Reading file: arching_back (14).mp4_13_keypoints.json
Reading file: arching_back (14).mp4_14_keypoints.json
Reading file: arching_back (14).mp4_15_keypoints.json
Reading file: arching_back (14).mp4_16_keypoints.json
Reading file: arching_back (14).mp4_17_keypoints.json
Reading file: arching_back (14).mp4_18_keypoints.json
Reading file: arching_back (14).mp4_19_keypoints.json
Reading file: arching_back (14).mp4_20_keypoints.json
Reading file: arching_back (14).mp4

Reading file: arching_back (21).mp4_38_keypoints.json
Reading file: arching_back (21).mp4_39_keypoints.json
Reading file: arching_back (21).mp4_40_keypoints.json
Reading file: arching_back (21).mp4_41_keypoints.json
Reading file: arching_back (21).mp4_42_keypoints.json
Reading file: arching_back (21).mp4_43_keypoints.json
Reading file: arching_back (21).mp4_44_keypoints.json
Reading file: arching_back (21).mp4_45_keypoints.json
Reading file: arching_back (21).mp4_46_keypoints.json
Reading file: head_banging (1).mp4_1_keypoints.json
Reading file: head_banging (1).mp4_2_keypoints.json
Reading file: head_banging (1).mp4_3_keypoints.json
Reading file: head_banging (1).mp4_4_keypoints.json
Reading file: head_banging (1).mp4_5_keypoints.json
Reading file: head_banging (1).mp4_6_keypoints.json
Reading file: head_banging (1).mp4_7_keypoints.json
'More than one detection in file, check the noise:'
head_banging (1).mp4_7_keypoints.json
Reading file: head_banging (1).mp4_8_keypoints.json
Reading 

Reading file: head_banging (5).mp4_136_keypoints.json
Reading file: head_banging (5).mp4_137_keypoints.json
Reading file: head_banging (5).mp4_138_keypoints.json
Reading file: head_banging (5).mp4_139_keypoints.json
Reading file: head_banging (5).mp4_140_keypoints.json
Reading file: head_banging (5).mp4_141_keypoints.json
Reading file: head_banging (5).mp4_142_keypoints.json
Reading file: head_banging (5).mp4_143_keypoints.json
Reading file: head_banging (5).mp4_144_keypoints.json
Reading file: head_banging (5).mp4_145_keypoints.json
Reading file: head_banging (5).mp4_146_keypoints.json
Reading file: head_banging (5).mp4_147_keypoints.json
Reading file: head_banging (5).mp4_148_keypoints.json
Reading file: head_banging (5).mp4_149_keypoints.json
Reading file: head_banging (5).mp4_150_keypoints.json
Reading file: head_banging (5).mp4_151_keypoints.json
Reading file: head_banging (5).mp4_152_keypoints.json
Reading file: head_banging (5).mp4_153_keypoints.json
Reading file: head_banging (

...

Reading file: sucking_fingers (22).mp4_142_keypoints.json
Reading file: sucking_fingers (22).mp4_143_keypoints.json
Reading file: sucking_fingers (22).mp4_144_keypoints.json
Reading file: sucking_fingers (22).mp4_145_keypoints.json
Reading file: sucking_fingers (22).mp4_146_keypoints.json
Reading file: sucking_fingers (22).mp4_147_keypoints.json
Reading file: sucking_fingers (22).mp4_148_keypoints.json
Reading file: sucking_fingers (22).mp4_149_keypoints.json
Reading file: sucking_fingers (22).mp4_150_keypoints.json
Reading file: sucking_fingers (22).mp4_151_keypoints.json
Reading file: sucking_fingers (22).mp4_152_keypoints.json
Reading file: sucking_fingers (22).mp4_153_keypoints.json
Reading file: sucking_fingers (22).mp4_154_keypoints.json
Reading file: sucking_fingers (22).mp4_155_keypoints.json
Reading file: sucking_fingers (22).mp4_156_keypoints.json
Reading file: sucking_fingers (22).mp4_157_keypoints.json
Reading file: sucking_fingers (22).mp4_158_keypoints.json
Reading file: 

Reading file: sucking_fingers (25).mp4_126_keypoints.json
Reading file: sucking_fingers (25).mp4_127_keypoints.json
Reading file: sucking_fingers (25).mp4_128_keypoints.json
Reading file: sucking_fingers (25).mp4_129_keypoints.json
Reading file: sucking_fingers (25).mp4_130_keypoints.json
Reading file: sucking_fingers (25).mp4_131_keypoints.json
Reading file: sucking_fingers (25).mp4_132_keypoints.json
Reading file: sucking_fingers (25).mp4_133_keypoints.json
Reading file: sucking_fingers (25).mp4_134_keypoints.json
Reading file: sucking_fingers (25).mp4_135_keypoints.json
Reading file: sucking_fingers (25).mp4_136_keypoints.json
Reading file: sucking_fingers (25).mp4_137_keypoints.json
Reading file: sucking_fingers (25).mp4_138_keypoints.json
Reading file: sucking_fingers (25).mp4_139_keypoints.json
Reading file: sucking_fingers (25).mp4_140_keypoints.json
Reading file: sucking_fingers (25).mp4_141_keypoints.json
Reading file: sucking_fingers (25).mp4_142_keypoints.json
Reading file: 

Reading file: sucking_fingers (28).mp4_4_keypoints.json
Reading file: sucking_fingers (28).mp4_5_keypoints.json
Reading file: sucking_fingers (28).mp4_6_keypoints.json
Reading file: sucking_fingers (28).mp4_7_keypoints.json
Reading file: sucking_fingers (28).mp4_8_keypoints.json
Reading file: sucking_fingers (28).mp4_9_keypoints.json
Reading file: sucking_fingers (28).mp4_10_keypoints.json
Reading file: sucking_fingers (28).mp4_11_keypoints.json
Reading file: sucking_fingers (28).mp4_12_keypoints.json
Reading file: sucking_fingers (28).mp4_13_keypoints.json
Reading file: sucking_fingers (28).mp4_14_keypoints.json
Reading file: sucking_fingers (28).mp4_15_keypoints.json
Reading file: sucking_fingers (28).mp4_16_keypoints.json
Reading file: sucking_fingers (28).mp4_17_keypoints.json
Reading file: sucking_fingers (28).mp4_18_keypoints.json
Reading file: sucking_fingers (28).mp4_19_keypoints.json
Reading file: sucking_fingers (28).mp4_20_keypoints.json
Reading file: sucking_fingers (28).mp

Reading file: sucking_fingers (29).mp4_30_keypoints.json
Reading file: sucking_fingers (29).mp4_31_keypoints.json
Reading file: sucking_fingers (29).mp4_32_keypoints.json
Reading file: sucking_fingers (29).mp4_33_keypoints.json
Reading file: sucking_fingers (29).mp4_34_keypoints.json
Reading file: sucking_fingers (29).mp4_35_keypoints.json
Reading file: sucking_fingers (29).mp4_36_keypoints.json
Reading file: sucking_fingers (29).mp4_37_keypoints.json
Reading file: sucking_fingers (29).mp4_38_keypoints.json
Reading file: sucking_fingers (29).mp4_39_keypoints.json
Reading file: sucking_fingers (29).mp4_40_keypoints.json
Reading file: sucking_fingers (29).mp4_41_keypoints.json
Reading file: sucking_fingers (29).mp4_42_keypoints.json
Reading file: sucking_fingers (29).mp4_43_keypoints.json
Reading file: sucking_fingers (29).mp4_44_keypoints.json
Reading file: sucking_fingers (29).mp4_45_keypoints.json
Reading file: sucking_fingers (29).mp4_46_keypoints.json
Reading file: sucking_fingers (

Reading file: sucking_fingers (32).mp4_101_keypoints.json
Reading file: sucking_fingers (32).mp4_102_keypoints.json
Reading file: sucking_fingers (32).mp4_103_keypoints.json
Reading file: sucking_fingers (32).mp4_104_keypoints.json
Reading file: sucking_fingers (32).mp4_105_keypoints.json
Reading file: sucking_fingers (32).mp4_106_keypoints.json
Reading file: sucking_fingers (32).mp4_107_keypoints.json
Reading file: sucking_fingers (32).mp4_108_keypoints.json
Reading file: sucking_fingers (32).mp4_109_keypoints.json
Reading file: sucking_fingers (32).mp4_110_keypoints.json
Reading file: sucking_fingers (32).mp4_111_keypoints.json
